In [1]:
import pandas as pd

dataset_name='selfdeploy24'

protocol='http'

if(protocol=='http'):
    key_fields=['User-Agent', 'Accept', 'Url', 'Accept-Encoding', 'Version', 'Connection']
if(protocol=='tls'):
    key_fields=['handshake_random_bytes', 'handshake_time', 'handshake_ciphersuites', 'handshake_sessionid', 'extensions_hash_algs', 'extensions_supported_groups']

# Read the CSV file
file_path = 'result_by_service/proj_still_unknown_'+protocol+'_'+dataset_name+'_with_clusterid.csv'
df = pd.read_csv(file_path)
print(df)

result_df = pd.DataFrame()

#Align the unknown-class probe indices
grouped = df.groupby('y_true_organization_name')
for org_name, group in grouped:
    if (org_name =='censys' or org_name=='driftnet'):
        file_path = 'unknown_probe_index/data_'+protocol+'/'+dataset_name+'/'+org_name+'.csv'
        df2 = pd.read_csv(file_path)
        index_list=group['index_original']
        filtered_df = df2.loc[index_list]
        filtered_df['ClusterNo']=list(group['ClusterNo'])
    else:
        file_path = 'unknown_probe_index/data_'+protocol+'/'+dataset_name+'/unknown.csv'
        df2 = pd.read_csv(file_path)
        index_list=group['index_original']
        filtered_df = df2.loc[index_list]
        filtered_df['ClusterNo']=list(group['ClusterNo'])

    result_df = pd.concat([result_df,filtered_df], ignore_index=True)

print(result_df)

grouped_df = result_df.groupby('ClusterNo')

for cluster_index, group in grouped_df:
    print(f"Group {cluster_index}:")
    print('ips',len(set(group['src_ip'])),'packets',len(group))
    df = group
    #删除平台标签
    if('plat' in df.columns):
        del df['plat']
    del df['src_ip']
    
    df=df.fillna('N/A')
    # Create a list to store the unique values and frequents each field value appears
    types=[]
    counts = []
    
    for col in df.columns:
        c = df[col].value_counts()
        name=c.index
        counts.append(list(c))
        types.append(list(name))
        
    type_with_count=[]
    for i in range(len(counts)):
        if(len(counts[i])>0):
            mini_list=[]
            for j in range(len(counts[i])):
                mini_list.append(str(types[i][j])+':'+format(counts[i][j]/len(df)*100, '.1f')+'%')
            type_with_count.append(mini_list)
        else:
            type_with_count.append([])

    column_names = df.columns
    df_stat = pd.DataFrame(type_with_count).T
    df_stat.columns=column_names
    

    none_columns = df_stat.columns[df.isnull().all()]
    print(none_columns)
    for i in none_columns:
        del df_stat[i]

    #save to csv file
    if(protocol=='http' or protocol=='tls'):
        df_stat.head(3)[key_fields].to_csv('cluster_statistic/'+protocol+'_'+'stat'+'_'+str(int(cluster_index))+'.csv', index=False)
    else:
        df_stat.head(3).to_csv('cluster_statistic/'+protocol+'_'+'stat'+'_'+str(int(cluster_index))+'.csv', index=False)
        

                                                   proj  packet_pred  y_true  \
0     [-0.09535259008407593, -0.8042677044868469, -0...           18      27   
1     [-0.09535259008407593, -0.8042677044868469, -0...           18      27   
2     [-0.09535259008407593, -0.8042677044868469, -0...           18      27   
3     [-0.09535259008407593, -0.8042677044868469, -0...           18      27   
4     [-0.09535259008407593, -0.8042677044868469, -0...           18      27   
...                                                 ...          ...     ...   
8691  [-0.1230463981628418, -0.8936273455619812, -1....           18      26   
8692  [-0.807263970375061, -0.5167971253395081, -0.7...            8      26   
8693  [-0.1230463981628418, -0.8936273455619812, -1....           18      26   
8694  [-0.1230463981628418, -0.8936273455619812, -1....           18      26   
8695  [-0.6206821203231812, 0.5270804762840271, -0.5...            4      26   

                  ip proto  index_origi

/tmp/ipykernel_1316416/757192647.py:30: DtypeWarning: Columns (26,37,40,41,43,64,67,68,71,72,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(file_path)
/tmp/ipykernel_1316416/757192647.py:30: DtypeWarning: Columns (26,37,40,41,43,64,67,68,71,72,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(file_path)
/tmp/ipykernel_1316416/757192647.py:30: DtypeWarning: Columns (26,37,40,41,43,64,67,68,71,72,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(file_path)


                                             User-Agent Accept-Encoding  \
0     Mozilla/5.0 (compatible; CensysInspect/1.1; +h...            gzip   
1                                    Go-http-client/1.1             NaN   
2     Mozilla/5.0 (iPhone; CPU iPhone os 15_4 like M...   gzip, deflate   
3     Mozilla/5.0 (compatible; CensysInspect/1.1; +h...            gzip   
4     Mozilla/5.0 (iPhone; CPU iPhone OS 13_7 like M...   gzip, deflate   
...                                                 ...             ...   
8691                                 Go-http-client/1.1             NaN   
8692  Mozilla/5.0 (compatible; InternetMeasurement/1...   gzip, deflate   
8693                                 Go-http-client/1.1             NaN   
8694                                 Go-http-client/1.1             NaN   
8695                                        curl/7.64.1             NaN   

                                                 Accept Connection  \
0                            